# Cluster the neighborhoods in the city of Toronto


Installing neccessary packages

In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


#### Step 01. Importing the packages

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### Step 02. Requesting data from the Wikipidea site

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [7]:
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

#### Step 03. Extract the wikitable with neighborhood data

In [8]:
wiki_table = soup.find('table')
#print(wiki_table.prettify())

In [35]:
table_rows = wiki_table.find_all('tr')
wiki_output=[]


for tr in table_rows:
    td=tr.find_all('td')
    wiki_data=[]
    for column in td:
        wiki_data.append(column.text.strip())
    wiki_output.append(wiki_data)

wiki_pd=pd.DataFrame(wiki_output, columns =['PostalCode','Borough','Neighborhood'])    

wiki_pd=wiki_pd.drop(wiki_pd.index[0])

wiki_pd = wiki_pd.replace('\n','', regex=True)

wiki_pd.head()



,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


In [36]:
wiki_pd.shape

(180, 3)

#### Step 04. Data Cleaning

In [37]:
wiki_pd = wiki_pd[wiki_pd.Borough !='Not assigned']

In [38]:
wiki_pd.shape

(103, 3)

In [39]:
wiki_pd = wiki_pd.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [41]:
wiki_pd = wiki_pd.replace('/',',', regex=True)

In [46]:
wiki_pd.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [43]:
wiki_pd['Neighborhood'].isna().sum()

0

In [53]:
for index, row in wiki_pd.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']


In [54]:
wiki_pd.shape

(103, 3)